Assignment 1:

𝑒𝑥𝑝𝑒𝑛𝑑𝑖𝑡𝑢𝑟𝑒=873 + 0.0012 * 𝑎𝑛𝑛𝑢𝑎𝑙_𝑖𝑛𝑐𝑜𝑚𝑒 + 0.00002 * 𝑎𝑛𝑛𝑢𝑎𝑙_𝑖𝑛𝑐𝑜𝑚𝑒^2 − 223.57 * ℎ𝑎𝑣𝑒_𝑘𝑖𝑑𝑠

With a constant of 873, we would expect an observation of 0 for 'annual income' and 'have kids' to have a value of 873 for expenditure. This is likely due to the fact that an individual still has to spend money on themselves for healthcare. The -223.57 coefficient likely means that those with children spend $223.57 less on average than those without, which seems strange, although could be explained by the treatment of 'kids' as categorical, rather than continuous, since the number of children per family is probably highly skewed towards 1. 

In [19]:
# Assignment 2:
import numpy as np, pandas as pd, matplotlib.pyplot as plt, warnings, statsmodels.formula.api as smf, statsmodels.api as sm
from sklearn import linear_model
from sklearn.preprocessing import StandardScaler, scale
from sklearn.decomposition import PCA
from sqlalchemy import create_engine
from scipy.stats.mstats import winsorize

# Visual Preference
%matplotlib inline
warnings.filterwarnings(action='ignore')

postgres_user = 'dsbc_student'
postgres_pw = '7*.8G9QH21'
postgres_host = '142.93.121.174'
postgres_port = '5432'
postgres_db = 'weatherinszeged'

engine = create_engine('postgresql://{}:{}@{}:{}/{}'.format(
    postgres_user, postgres_pw, postgres_host, postgres_port, postgres_db))

weather_df = pd.read_sql_query('select * from weatherinszeged', con=engine)
engine.dispose()
weather_df.head()
weather_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 96453 entries, 0 to 96452
Data columns (total 12 columns):
date                   96453 non-null datetime64[ns, UTC]
summary                96453 non-null object
preciptype             96453 non-null object
temperature            96453 non-null float64
apparenttemperature    96453 non-null float64
humidity               96453 non-null float64
windspeed              96453 non-null float64
windbearing            96453 non-null float64
visibility             96453 non-null float64
loudcover              96453 non-null float64
pressure               96453 non-null float64
dailysummary           96453 non-null object
dtypes: datetime64[ns, UTC](1), float64(8), object(3)
memory usage: 8.8+ MB


In [20]:
# Limit the outliers and drop non-numerical
weather_df = weather_df.select_dtypes(exclude=['datetime64[ns, UTC]'])
weather_df = weather_df.select_dtypes(exclude=['object'])

weather_df['temp_diff'] = weather_df['apparenttemperature'] - weather_df['temperature']

weather_df['w_humidity'] = winsorize(weather_df['humidity'], (0,0.10))
weather_df['w_windspeed'] = winsorize(weather_df['windspeed'], (0,0.10))
weather_df['w_pressure'] = winsorize(weather_df['pressure'], (0,0.10))
weather_df['w_apparenttemperature'] = winsorize(weather_df['apparenttemperature'], (0,0.10))
weather_df['w_temperature'] = winsorize(weather_df['temperature'], (0,0.10))
weather_df['w_temp_diff'] = winsorize(weather_df['temp_diff'], (0,0.10))
weather_df = weather_df.dropna()

In [21]:
Y = weather_df['w_temp_diff']
X = sm.add_constant(weather_df[['w_humidity', 'w_windspeed', 'w_pressure', 'w_apparenttemperature', 'w_temperature']])
results = sm.OLS(Y,X).fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:            w_temp_diff   R-squared:                       0.990
Model:                            OLS   Adj. R-squared:                  0.990
Method:                 Least Squares   F-statistic:                 1.940e+06
Date:                Mon, 09 Sep 2019   Prob (F-statistic):               0.00
Time:                        19:07:06   Log-Likelihood:                 37631.
No. Observations:               96453   AIC:                        -7.525e+04
Df Residuals:                   96447   BIC:                        -7.519e+04
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
=========================================================================================
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
const                    -0.3078      0.006    -51.683      0.000      -0.319      -0.296
w_humidity                0.3715      0.004    103.694      0.000       0.364       0.378
w_windspeed               0.0040      0.000     34.823      0.000       0.004       0.004
w_pressure             4.689e-06   4.52e-06      1.038      0.299   -4.17e-06    1.35e-05
w_apparenttemperature     1.0137      0.001   1918.140      0.000       1.013       1.015
w_temperature            -1.0161      0.001  -1680.785      0.000      -1.017      -1.015
==============================================================================
Omnibus:                    99900.266   Durbin-Watson:                   0.282
Prob(Omnibus):                  0.000   Jarque-Bera (JB):          6295419.448
Skew:                          -5.266   Prob(JB):                         0.00
Kurtosis:                      41.152   Cond. No.                     1.23e+04
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.23e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

It appears that only pressure is statistically significant. I would not expect that and suspect that there is something missing from the data cleaning process.

In [22]:
# Determine linearity of the model
w_pred = lrm_weather.predict(X)

# Plot estimated Y and X which should have a quadratic relationship
plt.scatter(Y, w_pred)
plt.xlabel('feature')
plt.ylabel('target')
plt.title('Linear Regression to Capture Quadratic Relationship')
plt.show()

ValueError: shapes (96453,6) and (7,) not aligned: 6 (dim 1) != 7 (dim 0)